In [57]:
import Pkg
Pkg.add("ARFFFiles")
Pkg.add("DataFrames")

Pkg.add("MLJBase")
Pkg.add("NearestNeighborModels")

   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.11/Project.toml`
  No Changes to `/opt/julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.11/Project.toml`
  No Changes to `/opt/julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.11/Project.toml`
  No Changes to `/opt/julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.11/Project.toml`
  No Changes to `/opt/julia/environments/v1.11/Manifest.toml`


In [53]:
using ARFFFiles, DataFrames

train = ARFFFiles.load(DataFrame, "ECG5000_TRAIN.arff")

train_data = Matrix(train[:, 1:end-1])
train_data .= (train_data .- maximum(train_data)) ./ (maximum(train_data) .- minimum(train_data))

train_label = train[:, end]

test = ARFFFiles.load(DataFrame, "ECG5000_TEST.arff")

test_data = Matrix(test[:, 1:end-1])
test_data .= (test_data .- maximum(test_data)) ./ (maximum(test_data) .- minimum(test_data))

test_label = test[:, end]

4500-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 ⋮
 "2"
 "2"
 "2"
 "4"
 "2"
 "2"
 "2"
 "4"
 "2"
 "2"
 "2"
 "2"

In [54]:
using NearestNeighborModels, MLJBase

# 转换数据为 MLJ 支持的格式
train_data = MLJ.table(train_data)
test_data = MLJ.table(test_data)

# 实例化模型
knnc = KNNClassifier(weights=Inverse())

# 包装为 MLJ 的 machine
knnc_mach = machine(knnc, train_data, train_label)

# 训练模型
fit!(knnc_mach)

[ Info: Training machine(KNNClassifier(K = 5, …), …).


trained Machine; caches model-specific representations of data
  model: KNNClassifier(K = 5, …)
  args: 
    1:	Source @410 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @029 ⏎ AbstractVector{Multiclass{5}}


In [55]:
# 使用模型对测试数据进行预测
predictions = predict_mode(knnc_mach, test_data)

4500-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 "1"
 ⋮
 "4"
 "2"
 "2"
 "4"
 "2"
 "4"
 "4"
 "2"
 "4"
 "2"
 "2"
 "4"

In [56]:
# 计算准确率
accuracy = sum(predictions .== test_label) / length(test_label)

# 输出准确率
println("Accuracy: $accuracy")

Accuracy: 0.9395555555555556
